# Azure Planetary Computer GeoCatalog Testing

This notebook tests authentication and basic operations with Microsoft Planetary Computer Pro GeoCatalog using Azure Client Secret credentials.

## Prerequisites
- Azure Service Principal with access to GeoCatalog
- `.env` file with credentials configured
- Required Python packages installed

## 1. Install Required Packages

Install the necessary Python packages for Azure authentication and API interactions.

In [ ]:
# Install required packages
%pip install python-dotenv azure-identity requests pystac-client

## 2. Import Libraries and Load Configuration

Import necessary libraries and load credentials from the `.env` file.

In [1]:
import os
import json
import requests
from datetime import datetime, timedelta
from dotenv import load_dotenv
from azure.identity import ClientSecretCredential, DefaultAzureCredential, DeviceCodeCredential

# Load environment variables from .env file
load_dotenv()

# Get credentials from environment
AZURE_CLIENT_ID = os.getenv('AZURE_CLIENT_ID')
AZURE_TENANT_ID = os.getenv('AZURE_TENANT_ID')
AZURE_CLIENT_SECRET = os.getenv('AZURE_CLIENT_SECRET')
GEOCATALOG_URL = os.getenv('GEOCATALOG_URL')

# Verify credentials are loaded
if not all([AZURE_CLIENT_ID, AZURE_TENANT_ID, AZURE_CLIENT_SECRET, GEOCATALOG_URL]):
    raise ValueError("Missing required environment variables. Please check your .env file.")

print("✓ Configuration loaded successfully")
print(f"  Client ID: {AZURE_CLIENT_ID[:8]}...")
print(f"  Tenant ID: {AZURE_TENANT_ID[:8]}...")
print(f"  GeoCatalog URL: {GEOCATALOG_URL}")

ValueError: Missing required environment variables. Please check your .env file.

## 3. Authenticate with Azure Client Secret

Create a credential object and obtain an access token for the GeoCatalog API.

In [2]:
# Create credential using client secret
credential = ClientSecretCredential(
    tenant_id=AZURE_TENANT_ID,
    client_id=AZURE_CLIENT_ID,
    client_secret=AZURE_CLIENT_SECRET
)

# 3b Authenticate with Browser (Optoinal)

In [2]:
credential = DeviceCodeCredential()


In [3]:

# GeoCatalog scope
GEOCATALOG_SCOPE = "https://geocatalog.spatio.azure.com/.default"
api_version = "2025-04-30-preview"

# Get access token
try:
    token = credential.get_token(GEOCATALOG_SCOPE)
    print("✓ Authentication successful!")
    print(f"  Token obtained at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"  Token expires at: {datetime.fromtimestamp(token.expires_on).strftime('%Y-%m-%d %H:%M:%S')}")
    
    # Create authorization header
    headers = {
        "Authorization": f"Bearer {token.token}",
        "Content-Type": "application/json"
    }
except Exception as e:
    print(f"✗ Authentication failed: {e}")
    raise

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code F5TRRWH77 to authenticate.


KeyboardInterrupt: 

## 4. List All Collections

Retrieve and display all STAC collections available in the GeoCatalog.

In [4]:
# Get all collections
collections_url = f"{GEOCATALOG_URL}/stac/collections"

try:
    response = requests.get(
        collections_url,
        headers=headers,
        params={"api-version": api_version}
    )
    response.raise_for_status()
    
    collections_data = response.json()
    collections = collections_data.get('collections', [])
    
    print(f"✓ Found {len(collections)} collection(s)\n")
    
    if collections:
        for idx, collection in enumerate(collections, 1):
            print(f"{idx}. Collection ID: {collection.get('id')}")
            print(f"   Title: {collection.get('title', 'N/A')}")
            print(f"   Description: {collection.get('description', 'N/A')[:100]}...")
            print(f"   License: {collection.get('license', 'N/A')}")
            print()
    else:
        print("No collections found in this GeoCatalog.")
        
except requests.exceptions.HTTPError as e:
    print(f"✗ HTTP Error: {e}")
    if response.text:
        print(f"Response: {response.text}")
except Exception as e:
    print(f"✗ Error: {e}")

✗ HTTP Error: 400 Client Error: Bad Request for url: https://skywatch-mpcpro.bqfjagf3hedyagcf.northcentralus.geocatalog.spatio.azure.com/stac/collections?api-version=2025-04-30-preview
Response: {"error":{"code":"BadRequestError","message":"Expecting Value: Line 1 Column 1 (Char 0)"}}


## 6. Get Specific Collection Details

Retrieve detailed information about a specific collection (if available).

In [ ]:
# Get a specific collection (replace with your collection ID)
# If you have collections, use one from the list above
collection_id = "your-collection-id"  # Update this with an actual collection ID

if collection_id != "your-collection-id" and collections:
    collection_url = f"{GEOCATALOG_URL}/stac/collections/{collection_id}"
    
    try:
        response = requests.get(
            collection_url,
            headers=headers,
            params={"api-version": api_version}
        )
        response.raise_for_status()
        
        collection = response.json()
        print(f"✓ Collection '{collection_id}' details:")
        print(json.dumps(collection, indent=2))
        
    except requests.exceptions.HTTPError as e:
        print(f"✗ HTTP Error: {e}")
        if response.text:
            print(f"Response: {response.text}")
    except Exception as e:
        print(f"✗ Error: {e}")
else:
    print("⚠ Please update 'collection_id' with an actual collection ID from your GeoCatalog.")
    if collections:
        print(f"Available collection IDs: {[c.get('id') for c in collections]}")

## 7. Search for Items

Use the STAC search endpoint to query for items across collections.

In [ ]:
# Search for items in the catalog
search_url = f"{GEOCATALOG_URL}/stac/search"

# Basic search query (you can modify parameters based on your needs)
search_query = {
    "limit": 10  # Limit results to 10 items
}

# If you have a specific collection, you can filter by it:
# search_query["collections"] = ["your-collection-id"]

try:
    response = requests.post(
        search_url,
        headers=headers,
        params={"api-version": api_version},
        json=search_query
    )
    response.raise_for_status()
    
    search_results = response.json()
    features = search_results.get('features', [])
    
    print(f"✓ Search completed. Found {len(features)} item(s)\n")
    
    if features:
        for idx, item in enumerate(features, 1):
            print(f"{idx}. Item ID: {item.get('id')}")
            print(f"   Collection: {item.get('collection')}")
            print(f"   Geometry Type: {item.get('geometry', {}).get('type', 'N/A')}")
            print(f"   Properties: {list(item.get('properties', {}).keys())[:5]}")
            print()
    else:
        print("No items found. This may be normal if your GeoCatalog is empty.")
        
except requests.exceptions.HTTPError as e:
    print(f"✗ HTTP Error: {e}")
    if response.text:
        print(f"Response: {response.text}")
except Exception as e:
    print(f"✗ Error: {e}")

## 8. Get API Conformance Classes

Check which STAC API conformance classes are supported by this GeoCatalog.

In [ ]:
# Get API conformance information
conformance_url = f"{GEOCATALOG_URL}/stac/conformance"

try:
    response = requests.get(
        conformance_url,
        headers=headers,
        params={"api-version": api_version}
    )
    response.raise_for_status()
    
    conformance = response.json()
    conforms_to = conformance.get('conformsTo', [])
    
    print(f"✓ API Conformance Classes ({len(conforms_to)}):")
    print()
    for idx, conformance_class in enumerate(conforms_to, 1):
        # Extract the conformance class name from the URL
        class_name = conformance_class.split('/')[-1] if '/' in conformance_class else conformance_class
        print(f"{idx}. {class_name}")
        print(f"   {conformance_class}")
        
except requests.exceptions.HTTPError as e:
    print(f"✗ HTTP Error: {e}")
    if response.text:
        print(f"Response: {response.text}")
except Exception as e:
    print(f"✗ Error: {e}")

## 9. Summary

This notebook has demonstrated:
1. ✓ Loading Azure credentials from environment variables
2. ✓ Authenticating with Azure using ClientSecretCredential
3. ✓ Obtaining an access token for the GeoCatalog API
4. ✓ Making authenticated requests to the STAC API
5. ✓ Retrieving the STAC API landing page
6. ✓ Listing all collections
7. ✓ Getting specific collection details
8. ✓ Searching for items
9. ✓ Checking API conformance classes

### Next Steps:
- Create collections using POST to `/stac/collections`
- Add items to collections using POST to `/stac/collections/{collection_id}/items`
- Configure ingestion sources for automated data ingestion
- Explore data visualization and tiling operations
- Generate SAS tokens for secure asset access